In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import glob
import tqdm
import awkward as ak
import boost_histogram as bh
import sys
import vector
import pickle
import scipy

import mplhep
mplhep.style.use(mplhep.style.CMS)

import sys
sys.path.append("../mlpf/")
from plotting.plot_utils import pid_to_text, EVALUATION_DATASET_NAMES, load_eval_data

In [ ]:
yvals, X, _ = load_eval_data(
    "../experiments/clic-hits_20230421_213012_921390.gpu1.local/evaluation"+
    "/epoch_18/clic_edm_ttbar_hits_pf/*.parquet",
    max_files=200)

In [ ]:
yvals["pred_cls_sm"] = ak.softmax(yvals["pred_cls"], axis=-1)

In [ ]:
msk1=yvals["gen_cls_id"]==2
msk2=yvals["pred_cls_id"]==2
msk3=yvals["cand_cls_id"]==2

In [ ]:
X[msk1][:, :, 10]

In [ ]:
X[msk1&msk2][:, :, 10]

In [ ]:
plt.hist2d(
    ak.to_numpy(ak.flatten(X[msk1])[:, 7]),
    ak.to_numpy(ak.flatten(yvals["pred_cls"][msk1][:, :, 2])), bins=100);

In [ ]:
b = np.logspace(-2,1,61)
plt.hist(ak.flatten(X[msk1])[:, 5], bins=b, histtype="step", lw=2)
plt.hist(ak.flatten(X[msk1&msk2])[:, 5], bins=b, histtype="step", lw=2)
plt.hist(ak.flatten(X[msk1&msk3])[:, 5], bins=b, histtype="step", lw=2)
plt.xscale("log")

In [ ]:
iev = 7
plt.scatter(
    X[X[:, :, 0]==1][iev, :, 2],
    np.arctan2(X[X[:, :, 0]==1][iev, :, 3], X[X[:, :, 0]==1][iev, :, 4]),
    s=X[X[:, :, 0]==1][iev, :, 5]
)

plt.scatter(
    X[X[:, :, 0]==2][iev, :, 2],
    np.arctan2(X[X[:, :, 0]==2][iev, :, 3], X[X[:, :, 0]==2][iev, :, 4]),
    s=X[X[:, :, 0]==2][iev, :, 5]
)


plt.scatter(
    yvals["gen_eta"][yvals["gen_cls_id"]!=0][iev],
    yvals["gen_phi"][yvals["gen_cls_id"]!=0][iev],
    s=yvals["gen_energy"][yvals["gen_cls_id"]!=0][iev]
)

plt.scatter(
    yvals["pred_eta"][yvals["pred_cls_id"]!=0][iev],
    yvals["pred_phi"][yvals["pred_cls_id"]!=0][iev],
    s=yvals["pred_energy"][yvals["pred_cls_id"]!=0][iev]
)

plt.xlim(-4,4)
plt.ylim(-4,4)

In [ ]:
iev = 8
plt.scatter(
    X[X[:, :, 0]==2][iev, :, 6],
    X[X[:, :, 0]==2][iev, :, 7],
    s=X[X[:, :, 0]==2][iev, :, 5]
)
plt.xlim(-4000, 4000)
plt.ylim(-4000, 4000)

In [ ]:
iev = 9
msk1 = X[:, :, 0]==2
colors = [
    "gray", "red", "green", "blue", "orange", "purple"
]
for icls in range(0,6):
    msk2 = yvals["gen_cls_id"]==icls
    mult = 5 if icls==0 else 50
    plt.scatter(
        X[msk1 & msk2][iev, :, 6],
        X[msk1 & msk2][iev, :, 7],
        s=mult*X[msk1 & msk2][iev, :, 5],
        c=colors[icls],
        label=icls,
        alpha=0.2 if icls==0 else 1.0
    )
plt.legend()
plt.xlim(-4000, 4000)
plt.ylim(-4000, 4000)

In [ ]:
for icls in range(0,6):
    msk2 = yvals["pred_cls_id"]==icls
    mult = 5 if icls==0 else 50
    plt.scatter(
        X[msk1 & msk2][iev, :, 6],
        X[msk1 & msk2][iev, :, 7],
        s=mult*X[msk1 & msk2][iev, :, 5],
        c=colors[icls],
        label=icls,
        alpha=0.2 if icls==0 else 1.0
    )
plt.legend()
plt.xlim(-4000, 4000)
plt.ylim(-4000, 4000)

In [ ]:
for icls in range(0,6):
    msk2 = yvals["gen_cls_id"]==icls
    mult = 5 if icls==0 else 50
    plt.scatter(
        X[msk1 & msk2][iev, :, 6],
        X[msk1 & msk2][iev, :, 8],
        s=mult*X[msk1 & msk2][iev, :, 5],
        c=colors[icls],
        label=icls,
        alpha=0.2 if icls==0 else 1.0
    )
plt.legend()
plt.xlim(-4000, 4000)
plt.ylim(-4000, 4000)

In [ ]:
for icls in range(0,6):
    msk2 = yvals["pred_cls_id"]==icls
    mult = 5 if icls==0 else 50
    plt.scatter(
        X[msk1 & msk2][iev, :, 6],
        X[msk1 & msk2][iev, :, 8],
        s=mult*X[msk1 & msk2][iev, :, 5],
        c=colors[icls],
        label=icls,
        alpha=0.2 if icls==0 else 1.0
    )
plt.legend()
plt.xlim(-4000, 4000)
plt.ylim(-4000, 4000)

In [ ]:
import plotly.express as px
import pandas

iev = 2

df = pandas.DataFrame()
df["px"] = X[msk1][iev, :, 6].to_numpy()
df["py"] = X[msk1][iev, :, 7].to_numpy()
df["pz"] = X[msk1][iev, :, 8].to_numpy()
df["energy"] = 10*X[msk1][iev, :, 5].to_numpy()
df["gen_cls_id"] = yvals["gen_cls_id"][msk1][iev].to_numpy()
df["cand_cls_id"] = yvals["cand_cls_id"][msk1][iev].to_numpy()
df["pred_cls_sm_0"] = yvals["pred_cls_sm"][msk1][iev, :, 0].to_numpy()
df["pred_cls_sm_1"] = yvals["pred_cls_sm"][msk1][iev, :, 1].to_numpy()
df["pred_cls_sm_2"] = yvals["pred_cls_sm"][msk1][iev, :, 2].to_numpy()
df["pred_cls_sm_3"] = yvals["pred_cls_sm"][msk1][iev, :, 3].to_numpy()
df["subdetector"] = X[msk1][iev, :, 10].to_numpy()

fig = px.scatter_3d(
    df,
    x='px',
    y='pz',
    z='py',
    color='cand_cls_id',
    size='energy',
    hover_data=["pred_cls_sm_0", "pred_cls_sm_1", "pred_cls_sm_2", "pred_cls_sm_3", "cand_cls_id", "gen_cls_id", "subdetector"],
    color_continuous_scale=px.colors.diverging.Spectral)
fig.show()